# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
weather_df = pd.read_csv('../output_data/cities_wo_index.csv')
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Emerald,-23.5333,148.1667,71.60,68,8,5.75,AU,1611505750
1,Mayo,38.8876,-76.5119,34.00,55,1,10.36,US,1611505750
2,Punta Arenas,-53.1500,-70.9167,62.60,36,0,14.97,CL,1611505605
3,Saint-Pierre,-21.3393,55.4781,80.60,74,20,5.75,RE,1611505750
4,Paracuru,-3.4100,-39.0306,82.00,71,100,21.99,BR,1611505750
...,...,...,...,...,...,...,...,...,...
586,Quatre Cocos,-20.2078,57.7625,80.60,78,40,3.44,MU,1611505818
587,Wasilla,61.5814,-149.4394,32.00,93,90,3.44,US,1611505838
588,Muriwai Beach,-36.8167,174.4500,64.00,85,33,11.65,NZ,1611505838
589,Birao,10.2849,22.7882,84.51,14,6,6.51,CF,1611505839


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
 # Configure gmaps
gmaps.configure(api_key=g_key)

In [26]:
locations = weather_df[['Lat',"Lng"]]
humidity = weather_df['Humidity']
m = gmaps.Map()
heatlayer = gmaps.heatmap_layer(locations,weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
# DF with Max Temperature lower than 85 but higher than 60

narrow_df = weather_df.loc[(weather_df["Max Temp"] < 85) & (weather_df["Max Temp"] > 60)].dropna()

narrow_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Emerald,-23.5333,148.1667,71.60,68,8,5.75,AU,1611505750
2,Punta Arenas,-53.1500,-70.9167,62.60,36,0,14.97,CL,1611505605
3,Saint-Pierre,-21.3393,55.4781,80.60,74,20,5.75,RE,1611505750
4,Paracuru,-3.4100,-39.0306,82.00,71,100,21.99,BR,1611505750
6,Port Alfred,-33.5906,26.8910,66.00,91,84,13.31,ZA,1611505751
...,...,...,...,...,...,...,...,...,...
584,Sawākin,19.1059,37.3321,76.89,77,31,10.67,SD,1611505838
586,Quatre Cocos,-20.2078,57.7625,80.60,78,40,3.44,MU,1611505818
588,Muriwai Beach,-36.8167,174.4500,64.00,85,33,11.65,NZ,1611505838
589,Birao,10.2849,22.7882,84.51,14,6,6.51,CF,1611505839


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df = narrow_df

In [40]:
hotel_df["Hotel Name"] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Emerald,-23.5333,148.1667,71.60,68,8,5.75,AU,1611505750,
2,Punta Arenas,-53.1500,-70.9167,62.60,36,0,14.97,CL,1611505605,
3,Saint-Pierre,-21.3393,55.4781,80.60,74,20,5.75,RE,1611505750,
4,Paracuru,-3.4100,-39.0306,82.00,71,100,21.99,BR,1611505750,
6,Port Alfred,-33.5906,26.8910,66.00,91,84,13.31,ZA,1611505751,
...,...,...,...,...,...,...,...,...,...,...
584,Sawākin,19.1059,37.3321,76.89,77,31,10.67,SD,1611505838,
586,Quatre Cocos,-20.2078,57.7625,80.60,78,40,3.44,MU,1611505818,
588,Muriwai Beach,-36.8167,174.4500,64.00,85,33,11.65,NZ,1611505838,
589,Birao,10.2849,22.7882,84.51,14,6,6.51,CF,1611505839,


In [42]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    #items within function we created in Activity 2
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(base_url, params=params)
    data = response.json()
    try:
        print(f'Working on {index} row')
        hotel_df.loc[index,"Hotel Name"] = data['results'][0]["name"]
    except:
        print(f'Missing value for row number {index}')

#Visualize Data
hotel_df

Working on 0 row
Working on 2 row
Working on 3 row
Working on 4 row
Working on 6 row
Working on 9 row
Working on 13 row
Working on 15 row
Working on 16 row
Working on 17 row
Missing value for row number 17
Working on 18 row
Working on 21 row
Missing value for row number 21
Working on 22 row
Working on 26 row
Working on 27 row
Working on 34 row
Working on 36 row
Working on 39 row
Missing value for row number 39
Working on 40 row
Working on 41 row
Working on 43 row
Working on 48 row
Working on 49 row
Working on 54 row
Missing value for row number 54
Working on 55 row
Working on 56 row
Working on 58 row
Working on 60 row
Working on 61 row
Working on 63 row
Missing value for row number 63
Working on 65 row
Working on 72 row
Working on 75 row
Working on 76 row
Working on 82 row
Working on 83 row
Working on 84 row
Working on 87 row
Working on 88 row
Working on 90 row
Working on 91 row
Missing value for row number 91
Working on 92 row
Working on 95 row
Missing value for row number 95
Working 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Emerald,-23.5333,148.1667,71.60,68,8,5.75,AU,1611505750,Stay On Sullivan
2,Punta Arenas,-53.1500,-70.9167,62.60,36,0,14.97,CL,1611505605,Hotel Dreams Del Estrecho
3,Saint-Pierre,-21.3393,55.4781,80.60,74,20,5.75,RE,1611505750,LODGE PALMAE
4,Paracuru,-3.4100,-39.0306,82.00,71,100,21.99,BR,1611505750,Pousada Oasis Paracuru
6,Port Alfred,-33.5906,26.8910,66.00,91,84,13.31,ZA,1611505751,Royal St Andrews Hotel
...,...,...,...,...,...,...,...,...,...,...
584,Sawākin,19.1059,37.3321,76.89,77,31,10.67,SD,1611505838,Arab Plaza Hotel عرب بلازا هوتيل
586,Quatre Cocos,-20.2078,57.7625,80.60,78,40,3.44,MU,1611505818,Shangri-La's Le Touessrok Resort & Spa
588,Muriwai Beach,-36.8167,174.4500,64.00,85,33,11.65,NZ,1611505838,216 Luxury Accommodation Auckland
589,Birao,10.2849,22.7882,84.51,14,6,6.51,CF,1611505839,


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))